In [ ]:
import pandas as pd
import numpy as np


df = pd.read_pickle('C:/Users/Rafael/OneDrive/Documentos/TCC Rafaela/Dados/Pickle/cancer_mama_regiao_sudeste_sem_nulos.pickle')


In [ ]:
# df['AP_CNSPCN'].head()

pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN', 'AP_PRIPAL'], 
                             values=['AP_DTINIC', 'AQ_ESTADI'], 
                             aggfunc=['min', 'max'])
pivot_table.head()

In [ ]:
#Calcular a diferença do estadiamento entre os períodos

pivot_table['dif_estadiamento'] = pivot_table[('max','AQ_ESTADI')]-pivot_table[('min','AQ_ESTADI')]

#Contar a qauntidade de cada valor
pivot_table['dif_estadiamento'].value_counts()

In [ ]:
pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN'], 
                             values=['AP_DTINIC', 'AQ_ESTADI'], 
                             aggfunc=['min', 'max'])

#Cálculo da Diferença entre o Máximo e Mínimo do Estadiamento
pivot_table['dif_estadiamento'] = pivot_table[('max','AQ_ESTADI')]-pivot_table[('min','AQ_ESTADI')]
pivot_table['dif_estadiamento'].value_counts()

In [ ]:
# Renomear as colunas da tabela dinâmica
pivot_table = pivot_table.set_axis(['inicio','estadiamento_inicio','fim','estadiamento_fim','dif_estadiamento'], axis=1)
pivot_table.head()

In [ ]:
# Merge da tabela pivotada e do dataset
df = pd.merge(df, pivot_table, on=['AP_CNSPCN'], how='left')
df.head()

In [ ]:
# Pivotar tabela
pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN', 'AP_PRIPAL'], 
                             values=['AP_DTINIC', 'AQ_ESTADI'], 
                             aggfunc=['min', 'max'])

# Calcular o total de tratamento realizados pelo paciente
total_tratamentos = pivot_table.reset_index().groupby('AP_CNSPCN')['AP_PRIPAL'].count()

#Fazer o merge dessa coluna de total de tratamentos ao dataset.
df = pd.merge(df, total_tratamentos, on=['AP_CNSPCN'], how='left')
df.head()


In [ ]:
# Renomeando as colunas
df = df.rename(columns={'AP_PRIPAL_x': 'AP_PRIPAL'})
df = df.rename(columns={'AP_PRIPAL_y': 'total_tratamentos'})
df.head()

In [ ]:
#Pivot da tabela
pivot_table = pd.pivot_table(df, 
                             index=['AP_CNSPCN', 'AP_PRIPAL'], 
                             values=['AP_DTINIC'], 
                             aggfunc=['min', 'max'])
procedimento_inicial = pivot_table.reset_index().groupby('AP_CNSPCN')['AP_PRIPAL'].first()

# Obtendo o Primeiro Procedimento por Paciente
df = pd.merge(df, procedimento_inicial, on=['AP_CNSPCN'], how='left')
df.head()

In [ ]:
#Renomear colunas 
df = df.rename(columns={'AP_PRIPAL_x': 'AP_PRIPAL'})
df = df.rename(columns={'AP_PRIPAL_y': 'procedimento_inicial'})
df.head()

In [ ]:
# Criação da coluna Piora
df['piora'] = df['dif_estadiamento'].apply(lambda x: 1 if x > 0 else 0)
df['piora'].value_counts()

#Indicador de óbito 
df['obito'] = df['AP_OBITO'].apply(lambda x: 1 if x > 0 else 0)
df['obito'].value_counts()

#Indicador de Alta 
df['alta'] = df['AP_ALTA'].apply(lambda x: 1 if x > 0 else 0)
df['alta'].value_counts()

#Indicador de Melhora
df['melhora'] = df['AP_ENCERR'].apply(lambda x: 1 if x > 0 else 0)
df['melhora'].value_counts()

In [ ]:
#Pivot da tabela por CNS
pivot_table = pd.pivot_table(df,
                             index='AP_CNSPCN',
                             values=['obito','piora','melhora','alta'],
                             aggfunc='max'
                             )
pivot_table.head()

In [ ]:
# Verificar desfecho do Paciente
def verifica_desfecho(obito, piora, melhora, alta):
    if(obito > 0 ):
        return 'Óbito'
    elif(piora > 0 ):
        return 'Piora'
    elif(melhora > 0):
        return 'Melhora'
    elif(alta > 0):
        return 'Alta'
    else:
        return 'SemDesfecho'

#Criar coluna com esse desfecho
df['desfecho'] = np.vectorize(verifica_desfecho)(df['obito'], df['piora'], df['melhora'], df['alta'])
df['desfecho'].value_counts()

In [ ]:
df.drop(columns=['AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA'], inplace=True)

df.head()

In [ ]:
pivot_table['desfecho'] = np.vectorize(verifica_desfecho)(pivot_table['obito'], pivot_table['piora'], pivot_table['melhora'], pivot_table['alta'])
pivot_table['desfecho'].value_counts()
df = pd.merge(df, pivot_table, on='AP_CNSPCN', how='left')

In [ ]:
# Renomear a coluna
df = df.rename(columns={'desfecho_y': 'desfecho'})
df.head()

In [ ]:
# Remover linhas sem desfecho
df = df.loc[df['desfecho'] != 'SemDesfecho',:]

In [ ]:
# Excluindo as colunas 'AQ_ESQU_P1' e 'AQ_ESQU_P2'
df.drop(columns=['piora_x',	'obito_x',	'alta_x',	'melhora_x',	'desfecho_x',	'alta_y',	'melhora_y',	'obito_y',	'piora_y'], inplace=True)

df.head()

In [ ]:
# Pivot da Tabela para pegar max e min da idade
pivot_table = pd.pivot_table(data=df,
                             index='AP_CNSPCN',
                             values=['AP_DTINIC', 'AP_NUIDADE'],
                             aggfunc=['min','max'])

#Renomeação das colunas
pivot_table = pivot_table.set_axis(['AP_DTINIC_min', 'idade_inicial', 'AP_DTINIC_max', 'idade_max'], axis=1)

#Merge com o dataset
df = pd.merge(df, pivot_table, on='AP_CNSPCN', how='left')
df.head()

In [ ]:
# Excluindo as colunas 
df.drop(columns=['AP_DTINIC_min',	'AP_DTINIC_max'], inplace=True)

df.head()